In [1]:
import numpy as np
import os
import re
import sys

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers as well as reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [3]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_sequences = len(dataX)

In [4]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [5]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [6]:
# load the network weights from best checkpoint
filename = "weights-improvement-20-1.9537.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [7]:
def sample(preds, temp=1.0):
    '''
    Helper function for sampling from softmax with different temperatures.
    
    Inputs:
    preds: output of softmax function
    temp: temperature to scale by
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [8]:
# set seed for emissions
seed = "shall i compare thee to a summer's day?\n"
seed_to_int = [char_to_int[char] for char in seed]

# generate emissions
pattern = seed_to_int
print(pattern)
temps = [0.1, 0.25, 0.75, 1.5]    
for temp in temps:    
    print('Temperature', temp)
    
    # generate characters
    newlines = 0
    while newlines < 14:
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)[0]

        # sample according to temperature
        idx = sample(prediction, temp)

        #index = np.argmax(prediction[0])
        result = int_to_char[idx]
        pattern.append(idx)
        pattern = pattern[1:len(pattern)]
        
        if result == '\n':
            newlines += 1
        
        # output result 
        sys.stdout.write(result)
        sys.stdout.flush()
    print()

[30, 19, 12, 23, 23, 1, 20, 1, 14, 26, 24, 27, 12, 29, 16, 1, 31, 19, 16, 16, 1, 31, 26, 1, 12, 1, 30, 32, 24, 24, 16, 29, 3, 30, 1, 15, 12, 36, 11, 0]
Temperature 0.1
the sorete thet hart soou thes hore thee toee,
and toeerene toel io the simes sieee
oo the siale toet io thee is ooe 

                   
then io the were the wirl whth wor aettede
toee ie mo thee then hoedsess tiee,
  then theerere tool the whrl well soeet sieee
thet soeet thee thee to thee iese bre frewe
the seeet of toee in thee iy soee,
                     
then io the were thet wit thes wile soeetere
thee theer srert tooe the toees soele toee soeesert siee,
  thet thaerere to thae, and thene oo hense

Temperature 0.25
thee 

nen thel the tiiek asd soeerer searee 
  thas ie then sene, and thee io thee  
                   

lo so thesl sine thyu siat that thee seee,
thee an thy soatee thet hoest sooee
thee io the prren theseet siade
ho the iose toee in then fore cenent,
whene hear tiat then toee ioed soee thet seee

KeyboardInterrupt: 